# Purpose 
The purpose of this notebook is to augment only the training data and avoid any data leakage into the real-world testing sample 

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 19.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.5 MB/s eta 0:00:00:00:0100:01


In [3]:
# load: 
import os 
import json
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
# import nlpaug.augmenter.word.context_word_embs as aug
# from sklearn.utils import shuffle

In [4]:
# read_csvs: 
train_path = '/home/ec2-user/SageMaker/Kartik_Amazon_Thesis/general_training_data'
rating_management_df = pd.read_csv(os.path.join(train_path,'rating_managment_explicit_general.csv'))
disgreement_df = pd.read_csv(os.path.join(train_path,'disagreement_with_ratings_general.csv'))

In [5]:
# merge the 2 datasets: 
merged_df = rating_management_df.merge(disgreement_df[['review_id','disagreement_with_ratings_ohe']], on= 'review_id', how='left')

In [6]:
def create_combined_label(rating_management_col, disagreement_rating_col): 
    if rating_management_col ==1 or disagreement_rating_col ==1: 
        return 1 
    return 0

In [7]:
merged_df['combine_tag']= merged_df.apply(lambda x: create_combined_label(x.rating_managment_explicit_ohe, x.disagreement_with_ratings_ohe), axis =1)

In [8]:
# train test split - split trainning into train and validation, take 10% as validation
train, test_df = train_test_split(merged_df, test_size= 0.3, random_state=42, stratify=merged_df['combine_tag'])
# train test split - split trainning into train and validation, take 10% as validation
train_df_1, val_df = train_test_split(train, test_size= 0.3, random_state=42, stratify=train['combine_tag'])

In [9]:
# now augmenting and creating of synthetic data on just the training dataset: 
train_df_1.combine_tag.value_counts()

0    5172
1     330
Name: combine_tag, dtype: int64

In [11]:
# save train_df_1 for augmentation: done on vscode
train_df_1.to_csv('combined_train_rating_disagreement.csv', index = False)

In [24]:
# read the augmented dataframe: 
# train_df_1 = pd.read_csv('augmented_training_data_rating_disagreement_combined.csv')
tf.random.set_seed(1234)
train_df_1 = pd.read_csv('1000_augmented_training_data_rating_disagreement_combined.csv')

train_df_1.combine_tag.value_counts()

0    5172
1    1330
Name: combine_tag, dtype: int64

In [11]:
# load the pre-trained models: 
from transformers import RobertaTokenizer, TFRobertaModel
with tf.device('/device:GPU:0'):
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = TFRobertaModel.from_pretrained("roberta-base")

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [25]:

class create_tensor_dataset(): 
    
    '''Purpose: Create a tensorflow dataset for model training
    df_to_convert: The dataframe of interest, that contains the 
    text columns
    text: The column within the dataframe containing the text 
    batch_size: int, batch size
    tokenizer: The tokenizer to be utilised
    labels: The class/label column within the dataframe
    '''
    
    def __init__(self, df_to_convert, text,target_col, batch_size, labels=None ): 
        self.df_to_convert = df_to_convert
        self.text = text # column containing the text (here chunked_data)
        self.target_col = target_col # y column of interest here label
        self.batch_size = batch_size
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.labels = labels # list of all the labels 
        
    def generate_training_data(self, ids, masks):
        for i, text in tqdm(enumerate(self.df_to_convert[self.text])):
            with tf.device('/device:GPU:0'):
                
                tokenized_text =  self.tokenizer.encode_plus(
                    text,# The sequence or batch of sequences to be encoded.
                    max_length=256, #Controls the maximum length to use by one of the truncation/padding parameters.
                    truncation=True, # Truncate to a maximum length specified with the argument max_length
                    padding='max_length', #pad to a maximum length specified with the argument max_length 
                    add_special_tokens=True,# encode the sequences with the special tokens relative to their model.
                    return_tensors='tf'   #return TensorFlow tf.constant objects.
                )
                ids[i, :] = tokenized_text.input_ids ##List of token ids to be fed to a model
                masks[i, :] = tokenized_text.attention_mask #List of indices specifying which tokens should be attended to by the model
        return ids, masks
    
    def ohe_label(self): 
         # convert label values to a list 
        return self.df_to_convert[self.target_col].values.tolist()

    def DatasetMapFunction(self, input_ids, attn_masks, labels):
        # create and return key-value pair of input ids, attention mask and the corresnponding labels
        return {
            'input_ids': input_ids,
            'attention_mask': attn_masks
        }, labels
    
    def tensor_data(self): 
        # input_ids array
        X_input_ids = np.zeros((len(self.df_to_convert), 256))
        # attention mask array
        X_attn_masks = np.zeros((len(self.df_to_convert), 256))
        # retrieve the input ids and attention masks
        X_input_ids, X_attn_masks = self.generate_training_data(X_input_ids, X_attn_masks)
        # creation of tensorflow dataset: 
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, self.ohe_label()))
        dataset = dataset.map(self.DatasetMapFunction)
        dataset = dataset.shuffle(10000).batch(self.batch_size, drop_remainder=True)
        return dataset

In [26]:
# creation of tensorflow dataset for training
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
train_tensor =create_tensor_dataset(train_df_1, 'full_review', 'combine_tag',8 )
train_dataset =train_tensor.tensor_data() 

6502it [00:08, 783.60it/s]


In [27]:
# validation tensor dataset: 
with tf.device('/device:GPU:0'):
    val_tensor =create_tensor_dataset(val_df, 'full_review', 'combine_tag', 8)
    val_dataset =val_tensor.tensor_data() 

2359it [00:03, 735.03it/s]


In [28]:

# defining 2 input layers for input_ids and attn_masks
with tf.device('/device:GPU:0'):
    input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
    attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
    bert_embds = model.roberta(input_ids, attention_mask=attn_masks)[1] # pooled output
    # intermediate_layer_1 = tf.keras.layers.Dropout(0.5, name="dropout")(bert_embds)# 0 -> activation layer (3D), 1 -> pooled output layer (2D)
    intermediate_layer = tf.keras.layers.Dense(1024, activation='relu', name='intermediate_layer')(bert_embds)
#     intermediate_layer_2 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer_2')(intermediate_layer)
    # intermediate_layer_2 = tf.keras.layers.Dropout(0.4, name="dropout2")(intermediate_layer)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(intermediate_layer) # sigmoid -> calcs probs of binary classes
    roberta_imdb = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
    roberta_imdb.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [29]:
# utilization of adam optimization 
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
# utilised binary cross entropy loss 
loss_func = tf.keras.losses.BinaryCrossentropy()
roberta_imdb.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])

# early stopping - administer val_loss: 
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience= 3,
    mode='min'
)

with tf.device('/device:GPU:0'):

    hist = roberta_imdb.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100, 
        callbacks =[custom_early_stopping]
    )

Epoch 1/100
812/812 [==============================] - 416s 470ms/step - loss: 0.0278 - accuracy: 0.9920 - val_loss: 0.1916 - val_accuracy: 0.9617
Epoch 2/100
812/812 [==============================] - 379s 467ms/step - loss: 0.0174 - accuracy: 0.9952 - val_loss: 0.1723 - val_accuracy: 0.9617
Epoch 3/100
812/812 [==============================] - 354s 436ms/step - loss: 0.0111 - accuracy: 0.9968 - val_loss: 0.2107 - val_accuracy: 0.9613
Epoch 4/100
812/812 [==============================] - 358s 441ms/step - loss: 0.0108 - accuracy: 0.9962 - val_loss: 0.2084 - val_accuracy: 0.9664
Epoch 5/100
812/812 [==============================] - 377s 464ms/step - loss: 0.0073 - accuracy: 0.9983 - val_loss: 0.2072 - val_accuracy: 0.9698
Epoch 6/100
812/812 [==============================] - 366s 451ms/step - loss: 0.0064 - accuracy: 0.9978 - val_loss: 0.2170 - val_accuracy: 0.9630


In [17]:
def prepare_data(input_text, tokenizer):
    with tf.device('/device:GPU:0'):
        
        token = tokenizer.encode_plus(
            input_text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        return {
            'input_ids': tf.cast(token.input_ids, tf.float64),
            'attention_mask': tf.cast(token.attention_mask, tf.float64)
        }

def make_predictions(model, input_text,threshold, label_list=None ): 
    with tf.device('/device:GPU:0'):
        processed_data = prepare_data(input_text, tokenizer)
        probs = model.predict(processed_data)
#         print(probs)
        if probs[0]> threshold: 
            return label_list[0]
        else: 
            return label_list[1]
#     return(probs[0])

In [30]:
# tt = test_df.head(20)
label_list= [1, 0]
test_df['result']= test_df.full_review.apply(lambda x: make_predictions(roberta_imdb, x, 0.5, label_list))

1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 45ms/step


...

1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 46ms/step


In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['combine_tag'], test_df['result']))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3167
           1       0.69      0.74      0.71       202

    accuracy                           0.96      3369
   macro avg       0.84      0.86      0.85      3369
weighted avg       0.97      0.96      0.97      3369



In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['combine_tag'], test_df['result']))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3167
           1       0.79      0.70      0.74       202

    accuracy                           0.97      3369
   macro avg       0.89      0.85      0.86      3369
weighted avg       0.97      0.97      0.97      3369



In [20]:
print('done')

done
